In [ ]:
import os
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext, messagebox
import speech_recognition as sr
import pyttsx3
import requests
import json
import tkinter.simpledialog as simpledialog
import sympy as sp
import random


# Section 1: Microphone and Recording Test
import sounddevice as sd
import soundfile as sf
import datetime


def test_microphone():
    samplerate = 44100  # Hertz
    duration = 5  # seconds
    filename = 'test_output.wav'

    print("Recording...")
    myrecording = sd.rec(int(samplerate * duration), samplerate=samplerate, channels=1)
    sd.wait()  # Wait until recording is finished
    sf.write(filename, myrecording, samplerate)
    print("Recording saved to", filename)

# Initialize the speech recognition and text-to-speech engines
recognizer = sr.Recognizer()
tts_engine = pyttsx3.init()

# Section 2: Verify Speech Recognition
def test_speech_recognition():
    recognizer = sr.Recognizer()
    filename = r'D:\python\test_output.wav'  # Update with the actual path to your audio file

    with sr.AudioFile(filename) as source:
        recognizer.adjust_for_ambient_noise(source)  # Adjust for ambient noise
        audio = recognizer.record(source)
    
    try:
        print("Recognizing...")
        query = recognizer.recognize_google(audio)
        print(f"Recognized: {query}")
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand that.")
    except sr.RequestError as e:
        print(f"Could not request results; {e}")


# Define intents
intents = {
    "greeting": ["hello mitra", "hi", "hey"],
    "farewell": ["bye", "goodbye", "see you"],
    "thanks": ["thank you", "thanks", "appreciate it"],
    "time": ["what time is it", "current time", "time", "samay"],
    "weather": ["weather", "how's the weather", "mausam"],
    "joke": ["tell me a joke", "joke", "chutkula"],
    "math": ["solve this math problem", "math problem", "math"],
    "currency": ["convert currency", "exchange currency", "exchange money", "currency exchange", "currency converter"],
    "trivia": ["trivia", "fact", "tell me something interesting", "interesting facts"],
    "quote": ["motivation", "quote", "inspire me", "statement", "motivational quote"],
    "exit": ["exit", "quit", "stop", "end chat"]
    # Add more intents as needed
}

def speak_text(text):
    """Convert text to speech."""
    tts_engine.say(text)
    tts_engine.runAndWait()

def listen():
    """Capture audio from the microphone and convert it to text."""
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        try:
            print("Listening...")
            audio = recognizer.listen(source)
            text = recognizer.recognize_google(audio)
            return text
        except sr.UnknownValueError:
            speak_text("Sorry, I did not understand that.")
            return "Sorry, I did not understand that."
        except sr.RequestError as e:
            speak_text(f"Could not request results; {e}")
            return f"Could not request results; {e}"
    return None


def recognize_intent(message):
    """Recognize the intent of the user's message."""
    message = message.lower()
    for intent, keywords in intents.items():
        if any(keyword in message for keyword in keywords):
            return intent
    return None

def handle_intent(intent):
    """Handle recognized intents with predefined responses."""
    if intent == "greeting":
        return "Hello! How can I assist you today?"
    elif intent == "farewell":
        return "Goodbye! Have a great day!"
    elif intent == "thanks":
        return "You're welcome!"
    elif intent == "time":
        current_time = datetime.datetime.now().strftime("%I:%M %p")
        return f"The current time is {current_time}."
    elif intent == "weather":
        return get_weather()
    elif intent == "joke":
        return get_joke()
    elif intent == "math":
        return solve_math_problem()
    elif intent == "trivia":
        return get_trivia()
    elif intent == "currency":
        return convert_currency()
    elif intent == "quote":
        return get_motivational_quote()
    elif intent == "exit":
        return "Exiting the chat. Have a nice day!"
    # Add more intent handling as needed
    return None



def listen_message():
    """Capture a voice message, convert it to text, and get a response."""
    user_input = listen()
    if user_input:
        chat_log.insert(tk.END, f"You: {user_input}\n")
        intent = recognize_intent(user_input)
        if intent:
            response = handle_intent(intent)
        else:
            response = "I didn't understand that. Please try again."
        chat_log.insert(tk.END, f"Chat Mitra: {response}\n")
        speak_text(response)
    else:
        messagebox.showwarning("Input Error", "Please enter a message.")



def preprocess_message(message):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text.lower())
    filtered_message = [w for w in word_tokens if not w in stop_words]
    return " ".join(filtered_message)

# Initialize in-memory storage for trivia facts and jokes
trivia_facts = [
    "Honey never spoils. Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still edible.",
    "Bananas are berries, but strawberries are not.",
    "The shortest war in history was between Britain and Zanzibar on August 27, 1896. Zanzibar surrendered after 38 minutes.",
    "A single teaspoon of honey represents the life work of 12 bees.",
    "The inventor of the Pringles can, Fred Baur, is buried in one.",
    "Octopuses have three hearts, nine brains, and blue blood.",
    "The Eiffel Tower can be 15 cm taller during the summer due to thermal expansion of the metal.",
    "The human nose can detect about one trillion different scents.",
    "There are more stars in the universe than grains of sand on all the beaches on Earth.",
    "A day on Venus is longer than a year on Venus.",
    "A group of flamingos is called a 'flamboyance'.",
    "Koalas have fingerprints that are so similar to humans that they can taint crime scenes.",
    "Wombat poop is cube-shaped to prevent it from rolling away.",
    "There is a species of jellyfish that is immortal. The Turritopsis dohrnii can revert back to its juvenile form after reaching adulthood.",
    "Some cats are allergic to humans."
]
jokes = [
    "Why don't scientists trust atoms? Because they make up everything!",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "Why don't skeletons fight each other? They don't have the guts.",
    "What do you call fake spaghetti? An impasta!",
    "Why was the math book sad? Because it had too many problems.",
    "I'm reading a book on anti-gravity. It's impossible to put down.",
    "Did you hear about the mathematician who’s afraid of negative numbers? He will stop at nothing to avoid them.",
    "Why do cows wear bells? Because their horns don’t work.",
    "Why did the bicycle fall over? Because it was two-tired!",
    "Why don't programmers like nature? It has too many bugs.",
    "santa (apni mummy se) – maa khushkhabari hai, hum do se teen ho gaye hain... maa – badhai ho beta, kya hua hai beta ya beti ? santa – na beta, aur na beti, maine dusri shadi kar li hai",
    "sanju apni girlfriend ke pita se milane gaya... ladki ka pita:- main nahi chahta ki meri beti apni puri jindagi ek murkh insaan ke saath gujaare....sanju :- bas uncle, isilie to main use yahan se le jaane aaya hoon de joote....de chappal...",
    
]

motivational_quotes = [
    "The only way to do great work is to love what you do. - Steve Jobs",
    "The best time to plant a tree was 20 years ago. The second best time is now. - Chinese Proverb",
    "It does not matter how slowly you go as long as you do not stop. - Confucius",
    "The future belongs to those who believe in the beauty of their dreams. - Eleanor Roosevelt",
    "You miss 100% of the shots you don't take. - Wayne Gretzky",
    "The best way to predict the future is to invent it. - Alan Kay",
    "The only limit to our realization of tomorrow is our doubts of today. - Franklin D. Roosevelt",
    "The only place where success comes before work is in the dictionary. - Vidal Sassoon",
    "Your time is limited, don't waste it living someone else's life. - Steve Jobs",
    "Don't watch the clock; do what it does. Keep going. - Sam Levenson",
]


# Function to fetch a random trivia fact
def get_trivia():
    if trivia_facts:
        return random.choice(trivia_facts)
    else:
        return "No trivia facts available. Please add some trivia facts first."

# Function to fetch a random joke
def get_joke():
    if jokes:
        return random.choice(jokes)
    else:
        return "No jokes available. Please add some jokes first."

# Function to add a new trivia fact
def add_trivia_fact():
    fact = simpledialog.askstring("Add Trivia Fact", "Enter the trivia fact:")
    if fact:
        trivia_facts.append(fact)
        messagebox.showinfo("Success", "Trivia fact added successfully!")

# Function to add a new joke
def add_joke():
    joke = simpledialog.askstring("Add Joke", "Enter the joke:")
    if joke:
        jokes.append(joke)
        messagebox.showinfo("Success", "Joke added successfully!")

# Function to solve math problems
def solve_math_problem():
    """Solve math problems entered via the GUI."""
    problem = simpledialog.askstring("Math Problem", "Enter the math problem (e.g., lcm 12 18, gcd 48 60, percentage 50 200, si 1000 5 2, ci 1000 5 2):")
    if not problem:
        return "No math problem entered. Please try again."

    try:
        tokens = problem.split()
        if tokens[0] == "lcm":
            nums = list(map(int, tokens[1:]))
            result = sp.lcm(*nums)
            response = f"The LCM of {nums} is {result}."
        elif tokens[0] == "gcd":
            nums = list(map(int, tokens[1:]))
            result = sp.gcd(*nums)
            response = f"The GCD of {nums} is {result}."
        elif tokens[0] == "percentage":
            part = float(tokens[1])
            whole = float(tokens[2])
            result = (part / whole) * 100
            response = f"The percentage is {result:.2f}%."
        elif tokens[0] == "Simple Interest":
            principal = float(tokens[1])
            rate = float(tokens[2])
            time = float(tokens[3])
            result = (principal * rate * time) / 100
            response = f"The Simple Interest is {result}."
        elif tokens[0] == "Compound Interest":
            principal = float(tokens[1])
            rate = float(tokens[2])
            time = float(tokens[3])
            result = principal * (pow((1 + rate / 100), time)) - principal
            response = f"The Compound Interest is {result}."
        else:
            result = sp.sympify(problem).evalf()
            response = f"The solution to the math problem is {result}."
    except Exception as e:
        response = f"Sorry, I couldn't solve that math problem. Error: {e}"

    return response


# Function to send a message and display the response in the GUI
def send_message():
    user_input = user_entry.get().strip()
    if user_input:
        chat_log.insert(tk.END, f"You: {user_input}\n")
        user_entry.delete(0, tk.END)

        if 'math' in user_input.lower():
            problem = simpledialog.askstring("Math Problem", "Enter the math problem:")
            if problem:
                response = solve_math_problem(problem)
            else:
                response = "No problem entered. Please try again."
        elif 'trivia' in user_input.lower():
            response = get_trivia()
        elif 'joke' in user_input.lower():
            response = get_joke()
        else:
            response = "I didn't understand that. Please try again."

        chat_log.insert(tk.END, f"Chat Mitra: {response}\n")
        speak_text(response)
    else:
        messagebox.showwarning("Input Error", "Please enter a message.")




def get_weather():
    api_key = "dd536312bae84ac3b4c2cd78340d9aac"
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    
    # Get the city name from the user via a dialog
    city_name = simpledialog.askstring("City", "Enter the city name:")
    if not city_name:
        return "No city entered. Please try again."

    complete_url = f"{base_url}appid={api_key}&q={city_name}"
    response = requests.get(complete_url)
    data = response.json()

    if data["cod"] != "404":
        main_data = data["main"]
        weather_data = data["weather"][0]
        temp = main_data["temp"] - 273.15  # Convert from Kelvin to Celsius
        pressure = main_data["pressure"]
        humidity = main_data["humidity"]
        description = weather_data["description"]

        response = f"The current temperature in {city_name} is {temp:.1f}°C with {humidity}% humidity and {pressure} hPa pressure. The weather is described as {description}."
    else:
        response = "City not found. Please try again."

    print(response)
    speak_text(response)

    return response

# Function to fetch a random motivational quote
def get_motivational_quote():
    if motivational_quotes:
        return random.choice(motivational_quotes)
    else:
        return "No motivational quotes available. Please add some quotes first."
# List of supported currencies

currencies = {
    "USD": "United States",
    "EUR": "Euro",
    "HKD": "Hong Kong",
    "JPY": "Japan",
    "CNY": "China",
    "AFN": "Afghanistan",
    "AUD": "Australia",
    "BDT": "Bangladesh",
    "BTN": "Bhutan",
    "GBP": "British Pound",
    "CAD": "Canada",
    "INR": "India",
    "IDR": "Indonesia",
    "MYR": "Malaysia",
    "MVR": "Maldives",
    "NPR": "Nepal",
    "PKR": "Pakistan",
    "RUB": "Russia",
    "SGD": "Singapore",
    "ZAR": "South Africa",
    "KRW": "South Korea",
    "LKR": "Sri Lanka",
    "AED": "UAE"
}
exchange_rates = {
                "USD": {"INR": 83.085, "EUR": 0.9219, "GBP": 0.785, "HKD": 7.807, "JPY": 156.92, "CNY": 7.245, "AFN": 71.82, "AUD": 1.507, "BDT": 117.35, "BTN": 83.065, "CAD": 1.3661, "IDR": 16.065, "MYR": 4.699, "MVR": 15.46, "NPR": 132.95, "PKR": 278.3, "RUB": 88.4475, "SGD": 1.349, "ZAR": 18.4553, "KRW": 1363.87, "LKR": 300.38, "AED": 3.673},
                "EUR": {"INR": 90.093, "USD": 1.0849, "GBP": 0.8515, "HKD": 8.4696, "JPY": 170.24, "CNY": 7.86, "AFN": 77.92, "AUD": 1.6346, "BDT": 127.306, "BTN": 90.115, "CAD": 1.482, "IDR": 17.365, "MYR": 5.098, "MVR": 16.77, "NPR": 144.230, "PKR": 301.948, "RUB": 96.228, "SGD": 1.464, "ZAR": 20.0228, "KRW": 1479.67, "LKR": 325.893, "AED": 3.985},
                "GBP": {"INR": 105.842, "USD": 1.2739, "EUR": 1.1744, "HKD": 9.947, "JPY": 199.95, "CNY": 9.23, "AFN": 91.51, "AUD": 1.92, "BDT": 149.51, "BTN": 105.84, "CAD": 1.740, "IDR": 20551.61, "MYR": 5.987, "MVR": 19.699, "NPR": 169.40, "PKR": 354.63, "RUB": 113.548, "SGD": 1.7197, "ZAR": 23.516, "KRW": 1738.335, "LKR": 382.758, "AED": 4.680},
                "INR": {"USD": 0.012, "EUR": 0.011, "GBP": 0.010, "HKD": 0.094, "JPY": 1.888, "CNY": 0.087, "AFN": 0.864, "AUD": 0.018, "BDT": 1.41, "BTN": 0.999, "CAD": 0.0164, "IDR": 194.09, "MYR": 0.0565, "MVR": 0.18, "NPR": 1.599, "PKR": 3.35, "RUB": 1064, "SGD": 0.016, "ZAR": 0.222, "KRW": 16.42, "LKR": 3.615, "AED": 0.0442},
                "HKD": {"USD": 0.1280, "INR": 10.646, "EUR": 0.118, "JPY": 17.84, "CNY": 0.90, "AFN": 11.20, "AUD": 0.19, "BDT": 13.95, "BTN": 9.55, "CAD": 0.17, "IDR": 1911.16, "MYR": 0.59, "MVR": 1.96, "NPR": 16.94, "PKR": 36.16, "RUB": 10.26, "SGD": 0.17, "ZAR": 2.48, "KRW": 168.85, "LKR": 40.68, "AED": 0.47},
                "JPY": {"USD": 0.0091 , "INR": 0.68 , "EUR": 0.0078, "HKD": 0.0707 , "CNY": 0.0585 , "AFN": 0.7085 , "AUD": 0.0125 , "BDT": 0.77 , "BTN": 0.68 , "CAD": 0.0115 , "IDR": 130.68 , "MYR": 0.0375 , "MVR": 0.14 , "NPR": 1.07 , "PKR": 1.56 , "RUB": 0.68 , "SGD": 0.0122 , "ZAR": 0.14 , "KRW": 10.74 , "LKR": 2.25, "AED": 0.0334},
                "CNY": {"USD": 0.138, "INR": 11.49, "EUR": 0.127},
                "AFN": {"USD": 0.0139, "INR": 0.157, "EUR": 0.0128},
                "AUD": {"USD": 0.664, "INR": 55.23, "EUR": 0.612},
                "BDT": {"USD": 0.0085, "INR": 0.708, "EUR": 0.00785},
                "BTN": {"USD": 0.0120, "INR": 1.0, "EUR": 0.01},
                "CAD": {"USD": 0.7322, "INR": 60.86, "EUR": 0.674},
                "IDR": {"USD": 0.000062, "INR": 0.00515, "EUR": 0.000057},
                "MYR": {"USD": 0.213, "INR": 17.69, "EUR": 0.196, "HKD": 1.66},
                "MVR": {"USD": 0.065, "INR": 5.37, "EUR": 0.059, "HKD": 0.5049},
                "NPR": {"USD": 0.0076, "INR": 0.63, "EUR": 0.0069, "HKD": 0.0587},
                "PKR": {"USD": 0.0036, "INR": 0.29, "EUR": 0.0033, "HKD": 0.028},
                "RUB": {"USD": 0.0113, "INR": 0.938, "EUR": 0.010, "HKD": 0.088},
                "SGD": {"USD": 0.741, "INR": 61.6, "EUR": 0.682, "HKD": 5.78},
                "ZAR": {"USD": 0.054, "INR": 4.52, "EUR": 0.050, "HKD": 0.424,},
                "KRW": {"USD": 0.000734, "INR": 0.061, "EUR": 0.00067, "HKD": 0.0057 },
                "LKR": {"USD": 0.0033, "INR": 0.276, "EUR": 0.0030, "HKD": 0.026},
                "AED": {"USD": 0.27, "INR": 22.63, "EUR": 0.25, "HKD": 2.12}
                # Add more exchange rates as needed
            }
    
def convert_currency():
    def get_currency_code(country_name):
        for code, name in currencies.items():
            if name == country_name:
                return code

    def calculate_conversion():
        from_currency_code = get_currency_code(from_currency.get())
        to_currency_code = get_currency_code(to_currency.get())
        amount = amount_entry.get()

        if not amount.replace('.', '', 1).isdigit():
            messagebox.showerror("Error", "Please enter a valid amount.")
            return

        amount = float(amount)
        if from_currency_code and to_currency_code:
            if from_currency_code in exchange_rates and to_currency_code in exchange_rates[from_currency_code]:
                converted_amount = amount * exchange_rates[from_currency_code][to_currency_code]
                response = f"{amount} {from_currency_code} is equal to {converted_amount:.2f} {to_currency_code}."
            else:
                response = f"Exchange rate from {from_currency_code} to {to_currency_code} is not available."
        else:
            response = "Please fill in all fields correctly."
        
        dialog.destroy()
        chat_log.insert(tk.END, f"Chat Mitra: {response}\n")
        speak_text(response)
        # Use text-to-speech to speak the result
        engine.say(response)
        engine.runAndWait()

    # Create a dialog window for selecting currencies and amount
    dialog = tk.Toplevel(root)
    dialog.title("Currency Converter")
    dialog.geometry("350x250")

    tk.Label(dialog, text="Convert from:").pack()
    from_currency = ttk.Combobox(dialog, values=list(currencies.values()))
    from_currency.pack()

    tk.Label(dialog, text="Convert to:").pack()
    to_currency = ttk.Combobox(dialog, values=list(currencies.values()))
    to_currency.pack()

    tk.Label(dialog, text="Amount:").pack()
    amount_entry = tk.Entry(dialog)
    amount_entry.pack()

    convert_button = tk.Button(dialog, text="Convert", command=calculate_conversion)
    convert_button.pack(pady=10)

# Function to initialize the text-to-speech engine
   
      

# Function to send a message and display the response in the GUI
def send_message():
    user_input = user_entry.get().strip()
    if user_input:
        chat_log.insert(tk.END, f"You: {user_input}\n")
        user_entry.delete(0, tk.END)

        intent = recognize_intent(user_input)
        if intent:
            response = handle_intent(intent)
        else:
            response = "I didn't understand that. Please try again."

        chat_log.insert(tk.END, f"Chat Mitra: {response}\n")
        speak_text(response)
    else:
        messagebox.showwarning("Input Error", "Please enter a message.")

# Set up the GUI
root = tk.Tk()
root.title("Chat Mitra")
root.geometry("550x550")

chat_log = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=25)
chat_log.pack(pady=10)

user_entry = tk.Entry(root, width=40)
user_entry.pack(pady=5)

send_button = tk.Button(root, text="Send", command=send_message)
send_button.pack(pady=5)

listen_button = tk.Button(root, text="🎤 Listen", command=listen_message)  # Voice recording button
listen_button.pack(pady=5)



root.mainloop()


The current temperature in iglas is 42.8°C with 7% humidity and 994 hPa pressure. The weather is described as clear sky.


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Scripts\Anaconda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\manee\AppData\Local\Temp\ipykernel_16160\1502470219.py", line 420, in calculate_conversion
    engine.say(response)
    ^^^^^^
NameError: name 'engine' is not defined


The current temperature in mathura is 38.7°C with 6% humidity and 992 hPa pressure. The weather is described as scattered clouds.


In [25]:
a6cbbda2e1e4421c813e83fcee107b15.4792dfcf2b523449

SyntaxError: invalid decimal literal (1335418550.py, line 1)